# Aircraft Turnaround Management

### Why:
Aircraft Turnaround Management refers to the physical process of preparing an aircraft for its next flight.
This process is crucial for every airline and should be optimized in every way possible to reduce downtimes, save costs and make customers happy with in time flights.<br>
One of the main problems of this process optimization is the lack of actual knowledge about the current performance of this process in the fleet.
This demo shows a way how to track different parts of the process with Computer Vision technology and predefined business rules.

## Define your application
This demo makes use of three main ideas:
### 1. Object Detection
We will detect objects in our camera image given a trained object detection model. All you have to do is to provide the list of objects the model should recognize.
### 2. Areas of Interest
Areas of Interest are special areas in your image that are crucial for your business process and where you want to count objects and apply business rules on.<br>
In this demo these areas are defined by polygons upfront but they could also be created dynamically via functions.
### 3. Business rules
Business rules define the process you want to analyze. They are written as simple functions that, given the object detection history, will return the process status.<br>
These rules can be as simple as in this example but can scale up to any complexity you need

### Note:
Even though the Open-Source logic is defined here, these functions could also be loaded directly from a SAS Micro Analytics Service that serves other applications as well.

In [ ]:
import esppy
import threading
import time
import websocket
import json
import numpy as np
import base64
import cv2

In [ ]:
### User variables
# Image input size
input_w = 1920
input_h = 1080
# Model input size
model_input_w = 416
model_input_h = 416
# Maximum number of objects to detect
max_number_objects = 50
# Maximum size of object history
objects_history_size = 100
# Dashboard size
dashboard_width = 520

# List of objects to detect
object_list = ['person','car','airplane','fuel_truck','stairway','baggage_truck','ramp_loader','tank_hose', 'rolling_stairway','bus', 'ground_power']

# Areas of Interest
# Define your areas of interest with the following schema:
# 1. Polygon-points in your image
# 2. should the polygon be drawn or not (only relevant for visualization)
# 3. color of the polygon (only relevant for visualization)
# 4. which objects to count in the area of interest
areas_of_interest = dict(aircraft_area=([[840,240],[840,900],[1150,900],[1150,240]],'visible',(255,128,255),['airplane','ground_power']),
                         fueling_area=([[200,330],[200,460],[650,530],[650,400]],'visible',(255,153,153),['tank_hose']),
                         stairway1=([[1150,550],[1360,750],[1360,930],[1150,730]],'visible',(0,204,153),['person']),
                         stairway2=([[1080,280],[1180,280],[1400,400],[1400,520],[1180,400],[1080,400]],'visible',(153,255,51),['person']),
                         baggage_area=([[100,600],[840,600],[840,950],[100,950]],'visible',(102,204,255),['person','ramp_loader','baggage_truck']))

# Define your business rules as functions that will return a string with (processname, status)
# 3 Examples are given

# Business Rule function: Aircraft Stationary
# Ex.: Count number of 'airplane' in area 'aircraft_area' -> process starts if the average count for the last 50 frames is 0.95
stationary =   '''def stationary(timestamp, area_class_count_history, avg_count=0.95, frame_range=50):
                    if len(area_class_count_history) < frame_range:
                        return ('AIRCRAFT', 'NA')
                    mean_aircraft_in_area = np.mean([hist_val['aircraft_area']['airplane'] for hist_val in area_class_count_history[-frame_range:]])
                    if (mean_aircraft_in_area > avg_count):
                        status = ('AIRCRAFT', 'STATIONARY')
                    else:
                        status = ('AIRCRAFT', 'NA')
                    return status'''

#Business Rule function: Ground Power
# Ex.: Count number of 'ground_power' in area 'aircraft_area' -> process starts if the average count for the last 20 frames is 0.8
ground_power = '''def ground_power(timestamp, area_class_count_history, avg_count=0.8, frame_range=20):
                    if len(area_class_count_history) < frame_range:
                        return ('GROUND POWER', 'STOPPED')
                    mean_ground_power = np.mean([hist_val['aircraft_area']['ground_power'] for hist_val in area_class_count_history[-frame_range:]])
                    if (mean_ground_power > avg_count):
                        status = ('GROUND POWER', 'IN PROGRESS')
                    else:
                        status = ('GROUND POWER', 'STOPPED')
                    return status'''

# Business Rule function: Fueling Status
# Ex.: count number of 'tank_hose' in area 'fueling_area' -> process starts if the average count for the last 10 frames is 0.8
fueling =    '''def fueling(timestamp, area_class_count_history, avg_count=0.8, frame_range=10):
                    if len(area_class_count_history) < frame_range:
                        return ('FUELING', 'STOPPED')
                    mean_tank_hose_count = np.mean([hist_val['fueling_area']['tank_hose'] for hist_val in area_class_count_history[-frame_range:]])
                    if mean_tank_hose_count > avg_count:
                        status = ('FUELING', 'IN PROGRESS')
                    else:
                        status = ('FUELING', 'STOPPED')
                    return status'''

# Business Rule function: Baggage Loading Status
# Ex.: Count number of 'baggage_truck' and 'ramp_loader' in area 'bagagge_area' -> process starts if the average count for the last 20 frames is 0.8
baggage_loading =    '''def baggage_loading(timestamp, area_class_count_history, avg_count=0.8, frame_range=20):
                            if len(area_class_count_history) < frame_range:
                                return ('BAGGAGE LOADING', 'STOPPED')
                            mean_baggage_truck_count = np.mean([hist_val['baggage_area']['baggage_truck'] for hist_val in area_class_count_history[-frame_range:]])
                            mean_baggage_ramp_loader_count = np.mean([hist_val['baggage_area']['ramp_loader'] for hist_val in area_class_count_history[-frame_range:]])
                            if (mean_baggage_truck_count > avg_count) and (mean_baggage_truck_count > avg_count):
                                status = ('BAGGAGE LOADING', 'IN PROGRESS')
                            else:
                                status = ('BAGGAGE LOADING', 'STOPPED')
                            return status'''

# Business Rule function: Boarding Status
# Ex.: Count number of 'persons' in areas 'stairway1' and 'stairway2' -> process starts if the average count for the last 20 frames is 2
boarding =   '''def boarding(timestamp, area_class_count_history, avg_count=1.5, frame_range=50):
                    if len(area_class_count_history) < frame_range:
                        return ('BOARDING', 'STOPPED')
                    mean_persons_on_stairway1 = np.mean([hist_val['stairway1']['person'] for hist_val in area_class_count_history[-frame_range:]])
                    mean_persons_on_stairway2 = np.mean([hist_val['stairway2']['person'] for hist_val in area_class_count_history[-frame_range:]])
                    mean_persons_on_stairways = np.mean([mean_persons_on_stairway1, mean_persons_on_stairway2])
                    if (mean_persons_on_stairway1 > avg_count) or (mean_persons_on_stairway2 > avg_count):
                        status = ('BOARDING', 'IN PROGRESS')
                    else:
                        status = ('BOARDING', 'STOPPED')
                    return status'''

# Put all business rules in a dictionary for lookup - will be provided to SAS Event Stream Processing
business_rules = dict(stationary=stationary, ground_power=ground_power, boarding=boarding, baggage_loading=baggage_loading, fueling=fueling)

### Project Setup
# CAS Adapter Setup -> Stream data into SAS Viya
cas_host = 'localhost'
cashost_port = 5570
esp_host = 'localhost'
esppubsub_port = 31416
esp_project_name = 'project_airport_turnaround_management'
esp_query_name = 'query_airport_turnaround_management'

esp_window = 'w_transp'
cas_library = 'public'
cas_table = 'turnaround_management_data'
cas_username = 'sasboot'
cas_password = 'sasdemo'

cmd = '/opt/sas/viya/home/SASEventStreamProcessingEngine/6.2/bin/dfesp_cas_adapter -C type=sub,cashostport={}:{},url=dfESP://{}:{}/{}/{}/{}?snapshot=true,caslib={},castable={},casusername={},caspassword={}'.format(cas_host, cashost_port, esp_host, esppubsub_port, esp_project_name, esp_query_name, esp_window, cas_library, cas_table, cas_username, cas_password)
cas_conn = esppy.connectors.AdapterSubscriber(command=cmd)

esp = esppy.ESP(hostname='http://localhost:9900')
esp_project = esp.create_project(esp_project_name, 
                                 n_threads=8, 
                                 pubsub='manual')

### Query1: Detect License Plates ###
esp_project.add_continuous_query(esp_query_name)

# Window: Receive full image
q1_vid_capture = esp.SourceWindow(schema=('id*:int64', 'timestamp:int64', 'airplane_id:string', 'framecounter:int64', 'image:blob'),
                                  index_type='empty', 
                                  insert_only=True,
                                  pubsub=False,
                                  name='w_image_input')
esp_project.add_window(q1_vid_capture, contquery=esp_query_name)

# Window: Resize image
q1_resize = esp.CalculateWindow(schema=('id*:int64', 'timestamp:int64', 'airplane_id:string', 'framecounter:int64', 'image:blob', 'image_resized:blob'),
                                algorithm='ImageProcessing', 
                                name='w_image_resized', 
                                function='resize',
                                height=model_input_h, 
                                width=model_input_w, 
                                input_map=dict(imageInput='image'), 
                                output_map=dict(imageOutput='image_resized'))
esp_project.add_window(q1_resize, contquery=esp_query_name)

# Window: Model Reader
q1_model_reader = esp.ModelReaderWindow(name='w_read_yolo_model')
esp_project.add_window(q1_model_reader, contquery='query_airport_turnaround_management')

# Window: Model Request
q1_model_request = esp.SourceWindow(schema=('req_id*:int64', 'req_key:string', 'req_val:string'),
                                 index_type='empty', 
                                 insert_only=True,name='w_deploy_model')
esp_project.add_window(q1_model_request, contquery=esp_query_name)

# Window: Model Score
def score_window_fields(number_objects):
    _field = "id*:int64,timestamp:int64,airplane_id:string,framecounter:int64,image:blob,image_resized:blob,_nObjects_:double,"
    for obj in range(0,number_objects):
        _field += "_Object" + str(obj) + "_:string,"
        _field += "_P_Object" + str(obj) + "_:double,"
        _field += "_Object" + str(obj) + "_x:double,"
        _field += "_Object" + str(obj) + "_y:double,"
        _field += "_Object" + str(obj) + "_width:double,"
        _field += "_Object" + str(obj) + "_height:double,"
    return _field[:-1]
q1_model_score = esp.ScoreWindow(schema=score_window_fields(max_number_objects),
                              name='w_score_image')
q1_model_score.add_offline_model(model_type='astore', 
                                 input_map=dict(_image_='image_resized'))
esp_project.add_window(q1_model_score, contquery=esp_query_name)

# Window: Python Window that executes functions for Areas of Interests and Business Rules
q1_wpython = esp.PythonHelper(schema=('id*:int64', 'timestamp:int64', 'airplane_id:string', 'framecounter:int64', 'scored_image:blob', 
                                      'fueling_start:int64','fueling_end:int64',
                                      'baggage_unloading_start:int64','baggage_unloading_end:int64', 
                                      'baggage_loading_start:int64','baggage_loading_end:int64', 
                                      'aircraft_stationary_start:int64','aircraft_stationary_end:int64',
                                      'unboarding_start:int64', 'unboarding_end:int64',
                                      'boarding_start:int64', 'boarding_end:int64',
                                      'ground_power_start:int64', 'ground_power_end:int64'), 
                           index_type='empty', 
                           produces_only_inserts=True, 
                           name='w_python',pubsub=True)
q1_wpython.add_mas_info('module_1', 'objectDetFunctions', 'w_score_image', code_file='./aircraft_turnaround_management.py', inits=dict(image_shape=(input_h,input_w,3),
                                                                                                                              objects_history_size=objects_history_size,
                                                                                                                              object_list=object_list,
                                                                                                                              areas_of_interest=areas_of_interest,
                                                                                                                              business_rules=business_rules,
                                                                                                                              dashboard_width=dashboard_width))
esp_project.add_window(q1_wpython, contquery=esp_query_name)

# Window: Filter Window
q1_filter = esp.FilterWindow(name='w_filter', 
                             index_type='empty',
                             pubsub=True)
q1_filter.set_expression(expr='timestamp==timestamp')
q1_filter.set_splitter_expr(expr='framecounter%10')
esp_project.add_window(q1_filter, contquery=esp_query_name)

# Window: Transpose Events for SAS Visual Analytics
q1_transp = esp.TransposeWindow(name='w_transp', mode='long', tag_name='TAG', tag_values='start,end', tags_included='fueling,baggage_unloading,baggage_loading,aircraft_stationary,unboarding,boarding,ground_power', pubsub=True)
esp_project.add_window(q1_transp, contquery=esp_query_name)

# Targets
q1_vid_capture.add_target(q1_resize, role='data')
q1_resize.add_target(q1_model_score, role='data')
q1_model_request.add_target(q1_model_reader, role='request')
q1_model_reader.add_target(q1_model_score, role='model')
q1_model_score.add_target(q1_wpython, role='data')
q1_wpython.add_target(q1_filter, role='data')
q1_filter.add_target(q1_transp, role='data')

# CAS Connector
q1_filter.add_connector(cas_conn)

esp.load_project(esp_project)

# Publisher: Send Model
q1_pub_model = q1_model_request.create_publisher(blocksize=1, rate=0, pause=0, dateformat='%Y%dT%H:%M:%S.%f', opcode='insert', format='csv')
q1_pub_model.send('i,n,1,"USEGPUESP","1"\n')
q1_pub_model.send('i,n,2,"ndevices","1"\n')
q1_pub_model.send('i,n,3,"action","load"\n')
q1_pub_model.send('i,n,4,"type","astore"\n')
q1_pub_model.send('i,n,5,"reference","/data/notebooks/deep_learning_examples/Aircraft Turnaround Management/Tiny-Yolov2.astore"\n')
q1_pub_model.send('i,n,6,,\n')
q1_pub_model.close()

# Publisher: Send Video
q1_pub_video = q1_vid_capture.create_publisher(blocksize=1, rate=0, pause=0, opcode='insert', format='csv')

esp_project

In [ ]:
# Class to publish videos to SAS Event Stream Processing
class video_pub():
    def __init__(self, publisher, video_file, video_quality=95):
        self.cap = cv2.VideoCapture(video_file)
        self.video_quality = video_quality
        self.pub = publisher
        self.frame_counter = 0
        threading.Thread(target=self.stream, daemon=True).start()
        print('Publisher started!')
        
    def stream(self):
        while True:
            self.frame_counter += 1
            ret, frame = self.cap.read()
            encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), self.video_quality]
            _, buffer = cv2.imencode('.jpg', frame, encode_param)
            encoded_string = base64.b64encode(buffer).decode()
            strToSend = 'i, n, {}, {}, {}, {}, {}, \n'.format(int(time.time()*100), int(time.time()), 'airplane1', self.frame_counter, encoded_string)
            self.pub.send(strToSend)
            
# Class to subscribe to SAS Event Stream Processing to receive scored images - optionally save to video
class video_sub():
    def __init__(self, window, save_to_file=True, filename='', video_fps=25, video_size=(1920,1080)):
        self.ws = websocket.WebSocketApp(window.subscriber_url+"?format=json&mode=streaming&pagesize=1&schema=false",
                                 on_message = self.on_message,
                                 on_error = self.on_error,
                                 on_close = self.on_close)
        self.ws.on_open = self.on_open
        self.save_to_file = save_to_file
        if save_to_file == True:
            fourcc = cv2.VideoWriter_fourcc(*"h264")
            self.out = cv2.VideoWriter(filename, fourcc, video_fps, video_size)
        threading.Thread(target=self.ws.run_forever, daemon=True).start()
        print('Subscriber Started!')
        return

    def on_message(self, message):
        try:
            data = json.loads(message)
            imageBufferBase64 = data['events'][0]['event']['scored_image']['scored_image']
            nparr = np.frombuffer(base64.b64decode(imageBufferBase64), dtype=np.uint8)
            frame = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
            if self.save_to_file == True:
                self.out.write(frame)
            cv2.imshow('frame',frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                cv2.destroyAllWindows()
        except Exception as e:
            print(e)

    def on_error(self, error):
        print(error)

    def on_close(self):
        print("Websocket closed!")

    def on_open(self):
        print('Websocket open!')

publisher = video_pub(publisher=q1_pub_video,
                      video_file='/data/notebooks/deep_learning_examples/Aircraft Turnaround Management/full_cutted3.mp4',
                      video_quality=95)
subscriber = video_sub(window=q1_wpython, 
                       save_to_file=True, 
                       filename='/data/notebooks/deep_learning_examples/Aircraft Turnaround Management/full_cutted3_scored.avi', 
                       video_fps=25, 
                       video_size=(input_w+dashboard_width,input_h)) #2855,1080